In [1]:
from __future__ import print_function
import numpy as np
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
img1 = np.loadtxt('imgs_1.txt')
img2 = np.loadtxt('imgs_2.txt')
img3 = np.loadtxt('imgs_3.txt')
img4 = np.loadtxt('imgs_4.txt')
img5 = np.loadtxt('imgs_5.txt')
img6 = np.loadtxt('imgs_6.txt')
img7 = np.loadtxt('imgs_7.txt')
ecfp = np.loadtxt('ECFPs.txt')

img=[img2,img3,img4,img5,img6,img7]
sample = img1
ecfps = ecfp
for i in range(6):
    sample = np.append(sample, img[i], axis=0)
    
for i in range(6):
    ecfps = np.append(ecfps, ecfp, axis=0)
    
sample_train = sample
sample_val = sample
sample_train = sample_train / sample.max()
sample_val = sample_val / sample.max()

In [3]:
test_imgs = np.loadtxt('P2-Test/imgs-test.txt')
test_ecfp = np.loadtxt('P2-Test/ECFPs-list.txt')

test_imgs = test_imgs / test_imgs.max()

In [4]:
label = np.arange(32).reshape(32,1)
labels = np_utils.to_categorical(label, 32)
a = np.arange(16).reshape(4,4)

In [4]:
class RBM:
  
  def __init__(self, num_visible, num_hidden):
    self.num_hidden = num_hidden
    self.num_visible = num_visible
    self.debug_print = True

    # Initialize a weight matrix, of dimensions (num_visible x num_hidden), using
    # a uniform distribution between -sqrt(6. / (num_hidden + num_visible))
    # and sqrt(6. / (num_hidden + num_visible)). One could vary the 
    # standard deviation by multiplying the interval with appropriate value.
    # Here we initialize the weights with mean 0 and standard deviation 0.1. 
    # Reference: Understanding the difficulty of training deep feedforward 
    # neural networks by Xavier Glorot and Yoshua Bengio
    np_rng = np.random.RandomState(1234)

    self.weights = np.asarray(np_rng.uniform(
			low=-0.1 * np.sqrt(6. / (num_hidden + num_visible)),
                       	high=0.1 * np.sqrt(6. / (num_hidden + num_visible)),
                       	size=(num_visible, num_hidden)))


    # Insert weights for the bias units into the first row and first column.
    self.weights = np.insert(self.weights, 0, 0, axis = 0)
    self.weights = np.insert(self.weights, 0, 0, axis = 1)

  def train(self, data, max_epochs = 1000, learning_rate = 0.1):
    """
    Train the machine.
    Parameters
    ----------
    data: A matrix where each row is a training example consisting of the states of visible units.    
    """

    num_examples = data.shape[0]

    # Insert bias units of 1 into the first column.
    data = np.insert(data, 0, 1, axis = 1)

    for epoch in range(max_epochs):      
      # Clamp to the data and sample from the hidden units. 
      # (This is the "positive CD phase", aka the reality phase.)
      pos_hidden_activations = np.dot(data, self.weights)      
      pos_hidden_probs = self._logistic(pos_hidden_activations)
      pos_hidden_probs[:,0] = 1 # Fix the bias unit.
      pos_hidden_states = pos_hidden_probs > np.random.rand(num_examples, self.num_hidden + 1)
      # Note that we're using the activation *probabilities* of the hidden states, not the hidden states       
      # themselves, when computing associations. We could also use the states; see section 3 of Hinton's 
      # "A Practical Guide to Training Restricted Boltzmann Machines" for more.
      pos_associations = np.dot(data.T, pos_hidden_probs)

      # Reconstruct the visible units and sample again from the hidden units.
      # (This is the "negative CD phase", aka the daydreaming phase.)
      neg_visible_activations = np.dot(pos_hidden_states, self.weights.T)
      neg_visible_probs = self._logistic(neg_visible_activations)
      neg_visible_probs[:,0] = 1 # Fix the bias unit.
      neg_hidden_activations = np.dot(neg_visible_probs, self.weights)
      neg_hidden_probs = self._logistic(neg_hidden_activations)
      # Note, again, that we're using the activation *probabilities* when computing associations, not the states 
      # themselves.
      neg_associations = np.dot(neg_visible_probs.T, neg_hidden_probs)

      # Update weights.
      self.weights += learning_rate * ((pos_associations - neg_associations) / num_examples)

      error = np.sum((data - neg_visible_probs) ** 2)
      if self.debug_print:
        print("Epoch %s: error is %s" % (epoch, error))

  def run_visible(self, data):
    """
    Assuming the RBM has been trained (so that weights for the network have been learned),
    run the network on a set of visible units, to get a sample of the hidden units.
    
    Parameters
    ----------
    data: A matrix where each row consists of the states of the visible units.
    
    Returns
    -------
    hidden_states: A matrix where each row consists of the hidden units activated from the visible
    units in the data matrix passed in.
    """
    
    num_examples = data.shape[0]
    
    # Create a matrix, where each row is to be the hidden units (plus a bias unit)
    # sampled from a training example.
    hidden_states = np.ones((num_examples, self.num_hidden + 1))
    
    # Insert bias units of 1 into the first column of data.
    data = np.insert(data, 0, 1, axis = 1)

    # Calculate the activations of the hidden units.
    hidden_activations = np.dot(data, self.weights)
    # Calculate the probabilities of turning the hidden units on.
    hidden_probs = self._logistic(hidden_activations)
    # Turn the hidden units on with their specified probabilities.
    hidden_states[:,:] = hidden_probs > np.random.rand(num_examples, self.num_hidden + 1)
    # Always fix the bias unit to 1.
    # hidden_states[:,0] = 1
  
    # Ignore the bias units.
    hidden_states = hidden_states[:,1:]
    return hidden_states
    
  # TODO: Remove the code duplication between this method and `run_visible`?
  def run_hidden(self, data):
    """
    Assuming the RBM has been trained (so that weights for the network have been learned),
    run the network on a set of hidden units, to get a sample of the visible units.
    Parameters
    ----------
    data: A matrix where each row consists of the states of the hidden units.
    Returns
    -------
    visible_states: A matrix where each row consists of the visible units activated from the hidden
    units in the data matrix passed in.
    """

    num_examples = data.shape[0]

    # Create a matrix, where each row is to be the visible units (plus a bias unit)
    # sampled from a training example.
    visible_states = np.ones((num_examples, self.num_visible + 1))

    # Insert bias units of 1 into the first column of data.
    data = np.insert(data, 0, 1, axis = 1)

    # Calculate the activations of the visible units.
    visible_activations = np.dot(data, self.weights.T)
    # Calculate the probabilities of turning the visible units on.
    visible_probs = self._logistic(visible_activations)
    # Turn the visible units on with their specified probabilities.
    visible_states[:,:] = visible_probs > np.random.rand(num_examples, self.num_visible + 1)
    # Always fix the bias unit to 1.
    # visible_states[:,0] = 1

    # Ignore the bias units.
    visible_states = visible_states[:,1:]
    return visible_states
    
  def daydream(self, num_samples):
    """
    Randomly initialize the visible units once, and start running alternating Gibbs sampling steps
    (where each step consists of updating all the hidden units, and then updating all of the visible units),
    taking a sample of the visible units at each step.
    Note that we only initialize the network *once*, so these samples are correlated.
    Returns
    -------
    samples: A matrix, where each row is a sample of the visible units produced while the network was
    daydreaming.
    """

    # Create a matrix, where each row is to be a sample of of the visible units 
    # (with an extra bias unit), initialized to all ones.
    samples = np.ones((num_samples, self.num_visible + 1))

    # Take the first sample from a uniform distribution.
    samples[0,1:] = np.random.rand(self.num_visible)

    # Start the alternating Gibbs sampling.
    # Note that we keep the hidden units binary states, but leave the
    # visible units as real probabilities. See section 3 of Hinton's
    # "A Practical Guide to Training Restricted Boltzmann Machines"
    # for more on why.
    for i in range(1, num_samples):
      visible = samples[i-1,:]

      # Calculate the activations of the hidden units.
      hidden_activations = np.dot(visible, self.weights)      
      # Calculate the probabilities of turning the hidden units on.
      hidden_probs = self._logistic(hidden_activations)
      # Turn the hidden units on with their specified probabilities.
      hidden_states = hidden_probs > np.random.rand(self.num_hidden + 1)
      # Always fix the bias unit to 1.
      hidden_states[0] = 1

      # Recalculate the probabilities that the visible units are on.
      visible_activations = np.dot(hidden_states, self.weights.T)
      visible_probs = self._logistic(visible_activations)
      visible_states = visible_probs > np.random.rand(self.num_visible + 1)
      samples[i,:] = visible_states

    # Ignore the bias units (the first column), since they're always set to 1.
    return samples[:,1:]        
      
  def _logistic(self, x):
    return 1.0 / (1 + np.exp(-x))

In [7]:
sample.max()

4080256.0

In [10]:
if __name__ == '__main__':
    r = RBM(num_visible = 720, num_hidden = 128)
    training_data = sample_train
    r.train(training_data, max_epochs = 50000)
    print(r.weights)
    test = test_imgs
    print(r.run_visible(test)) 
    latent = r.run_visible(test)
    visible = r.run_hidden(test_ecfp)

Epoch 0: error is 39439.2687272
Epoch 1: error is 5118.16636377
Epoch 2: error is 4643.78617434
Epoch 3: error is 4110.85990035
Epoch 4: error is 3774.31565594
Epoch 5: error is 3301.29408199
Epoch 6: error is 2780.55241009
Epoch 7: error is 2333.09965974
Epoch 8: error is 2048.87191306
Epoch 9: error is 1696.42113976
Epoch 10: error is 1435.15933897
Epoch 11: error is 1166.07728933
Epoch 12: error is 991.203459226
Epoch 13: error is 807.862403917
Epoch 14: error is 667.093042618
Epoch 15: error is 569.571480396
Epoch 16: error is 475.126426941
Epoch 17: error is 423.946473548
Epoch 18: error is 351.665618623
Epoch 19: error is 338.079141564
Epoch 20: error is 327.426430564
Epoch 21: error is 234.043024032
Epoch 22: error is 245.717645765
Epoch 23: error is 243.777428151
Epoch 24: error is 229.32178106
Epoch 25: error is 213.198126305
Epoch 26: error is 188.916013921
Epoch 27: error is 164.812730087
Epoch 28: error is 209.432708904
Epoch 29: error is 194.242788056
Epoch 30: error is 17

Epoch 252: error is 74.6955666192
Epoch 253: error is 89.7913791284
Epoch 254: error is 78.9799195413
Epoch 255: error is 73.5893197194
Epoch 256: error is 86.5125475777
Epoch 257: error is 78.4570425906
Epoch 258: error is 82.6946421
Epoch 259: error is 73.9257772896
Epoch 260: error is 81.30878296
Epoch 261: error is 90.6600147292
Epoch 262: error is 82.6149928969
Epoch 263: error is 80.2121787101
Epoch 264: error is 75.992436214
Epoch 265: error is 68.0346186947
Epoch 266: error is 88.6793096002
Epoch 267: error is 83.7437932936
Epoch 268: error is 81.7302302102
Epoch 269: error is 73.5298722469
Epoch 270: error is 80.236269767
Epoch 271: error is 74.4891439716
Epoch 272: error is 85.4808597964
Epoch 273: error is 81.1580004485
Epoch 274: error is 83.7170159128
Epoch 275: error is 73.4595207787
Epoch 276: error is 77.61980475
Epoch 277: error is 75.7359164101
Epoch 278: error is 92.5752659852
Epoch 279: error is 81.6302656981
Epoch 280: error is 75.4455982426
Epoch 281: error is 97.

Epoch 494: error is 38.6607895668
Epoch 495: error is 42.4516578481
Epoch 496: error is 49.6153472961
Epoch 497: error is 42.9342309491
Epoch 498: error is 50.9703937481
Epoch 499: error is 39.1768233247
Epoch 500: error is 46.3416522992
Epoch 501: error is 40.5625912754
Epoch 502: error is 38.8568937357
Epoch 503: error is 49.7692401095
Epoch 504: error is 34.0402009959
Epoch 505: error is 37.2587295612
Epoch 506: error is 43.6710802555
Epoch 507: error is 55.8809475751
Epoch 508: error is 37.4695478466
Epoch 509: error is 37.5649179811
Epoch 510: error is 37.2312377501
Epoch 511: error is 39.3398561853
Epoch 512: error is 39.8494798168
Epoch 513: error is 40.2900322679
Epoch 514: error is 38.6525326232
Epoch 515: error is 39.6109576701
Epoch 516: error is 33.1544355773
Epoch 517: error is 36.7055400959
Epoch 518: error is 33.528034975
Epoch 519: error is 36.6309721191
Epoch 520: error is 33.7975914502
Epoch 521: error is 38.0928764124
Epoch 522: error is 35.6121174002
Epoch 523: erro

Epoch 743: error is 30.3488300529
Epoch 744: error is 31.0271197164
Epoch 745: error is 31.2649001877
Epoch 746: error is 40.481486557
Epoch 747: error is 28.55030133
Epoch 748: error is 28.1205254431
Epoch 749: error is 36.0889904482
Epoch 750: error is 32.0344142284
Epoch 751: error is 28.9813768049
Epoch 752: error is 28.2118823576
Epoch 753: error is 35.3134859407
Epoch 754: error is 38.18702319
Epoch 755: error is 31.2716206601
Epoch 756: error is 29.0797524477
Epoch 757: error is 30.1207825167
Epoch 758: error is 32.9775688277
Epoch 759: error is 30.8099665113
Epoch 760: error is 28.7899979126
Epoch 761: error is 30.6601241427
Epoch 762: error is 30.0935320271
Epoch 763: error is 28.8493791459
Epoch 764: error is 27.2845691331
Epoch 765: error is 29.6521080763
Epoch 766: error is 29.9907113822
Epoch 767: error is 28.0705297389
Epoch 768: error is 27.5943343886
Epoch 769: error is 26.2207568208
Epoch 770: error is 28.2912101093
Epoch 771: error is 30.8992452128
Epoch 772: error is

Epoch 1001: error is 24.4592080924
Epoch 1002: error is 28.0148504016
Epoch 1003: error is 29.4485781487
Epoch 1004: error is 35.3684764373
Epoch 1005: error is 24.9075373234
Epoch 1006: error is 27.5403275469
Epoch 1007: error is 28.199634485
Epoch 1008: error is 28.630676444
Epoch 1009: error is 29.643688586
Epoch 1010: error is 28.7658300467
Epoch 1011: error is 30.9563551656
Epoch 1012: error is 25.8889890171
Epoch 1013: error is 31.7356118871
Epoch 1014: error is 25.5940612754
Epoch 1015: error is 30.4621939339
Epoch 1016: error is 25.7096009961
Epoch 1017: error is 25.7193349632
Epoch 1018: error is 26.7780166644
Epoch 1019: error is 24.7846384693
Epoch 1020: error is 24.9250478766
Epoch 1021: error is 30.6202536883
Epoch 1022: error is 25.7462955506
Epoch 1023: error is 23.3850945083
Epoch 1024: error is 27.5291502987
Epoch 1025: error is 33.2475771618
Epoch 1026: error is 30.0375142469
Epoch 1027: error is 25.6948809336
Epoch 1028: error is 30.0652213618
Epoch 1029: error is 26

...

Epoch 48906: error is 10.510031126
Epoch 48907: error is 9.16108087196
Epoch 48908: error is 9.47617131125
Epoch 48909: error is 9.86520362554
Epoch 48910: error is 9.70713370608
Epoch 48911: error is 11.4662255677
Epoch 48912: error is 8.79042537091
Epoch 48913: error is 9.28439947452
Epoch 48914: error is 9.96664141792
Epoch 48915: error is 8.48898526104
Epoch 48916: error is 10.5417338949
Epoch 48917: error is 10.1030694297
Epoch 48918: error is 11.0724111202
Epoch 48919: error is 9.40774873289
Epoch 48920: error is 9.16518750307
Epoch 48921: error is 10.3154996366
Epoch 48922: error is 8.69000264978
Epoch 48923: error is 10.2873919217
Epoch 48924: error is 8.97063652032
Epoch 48925: error is 10.5911471974
Epoch 48926: error is 9.49689672199
Epoch 48927: error is 9.83711736558
Epoch 48928: error is 8.30049793962
Epoch 48929: error is 11.8611398136
Epoch 48930: error is 11.0245456713
Epoch 48931: error is 9.09629520728
Epoch 48932: error is 10.9844020935
Epoch 48933: error is 13.1286

Epoch 49141: error is 13.4762776462
Epoch 49142: error is 8.5720486331
Epoch 49143: error is 10.5175562304
Epoch 49144: error is 7.75833845756
Epoch 49145: error is 10.3678690916
Epoch 49146: error is 9.87973157498
Epoch 49147: error is 9.51136896982
Epoch 49148: error is 8.95992966161
Epoch 49149: error is 8.80065353614
Epoch 49150: error is 12.2179784244
Epoch 49151: error is 10.2525516478
Epoch 49152: error is 9.6606195194
Epoch 49153: error is 8.67714621547
Epoch 49154: error is 11.5963429429
Epoch 49155: error is 8.77903864644
Epoch 49156: error is 8.47499705462
Epoch 49157: error is 9.70341080224
Epoch 49158: error is 8.11903838891
Epoch 49159: error is 14.1985454826
Epoch 49160: error is 9.91403608598
Epoch 49161: error is 11.4426832774
Epoch 49162: error is 9.88378169314
Epoch 49163: error is 12.3863627333
Epoch 49164: error is 10.3941452408
Epoch 49165: error is 9.67528246901
Epoch 49166: error is 8.78239560224
Epoch 49167: error is 10.3462449756
Epoch 49168: error is 10.66486

Epoch 49377: error is 10.6420702321
Epoch 49378: error is 12.1674260255
Epoch 49379: error is 13.2426326084
Epoch 49380: error is 9.66250004955
Epoch 49381: error is 10.7923034378
Epoch 49382: error is 9.8217617317
Epoch 49383: error is 7.84735532165
Epoch 49384: error is 11.0451387028
Epoch 49385: error is 10.3554890655
Epoch 49386: error is 8.74002493627
Epoch 49387: error is 9.54622034451
Epoch 49388: error is 10.6512131229
Epoch 49389: error is 7.80711131516
Epoch 49390: error is 12.4885770508
Epoch 49391: error is 9.35271976621
Epoch 49392: error is 8.920269097
Epoch 49393: error is 12.4651709917
Epoch 49394: error is 9.42814220226
Epoch 49395: error is 9.04264963983
Epoch 49396: error is 11.1182689862
Epoch 49397: error is 8.71977624035
Epoch 49398: error is 10.4379793524
Epoch 49399: error is 10.7647848781
Epoch 49400: error is 10.5675012714
Epoch 49401: error is 9.19803214791
Epoch 49402: error is 10.6368515693
Epoch 49403: error is 10.4826901422
Epoch 49404: error is 9.7193444

Epoch 49613: error is 9.3864167807
Epoch 49614: error is 10.1550466712
Epoch 49615: error is 9.52576600066
Epoch 49616: error is 10.1373550363
Epoch 49617: error is 13.0044081849
Epoch 49618: error is 11.8434230171
Epoch 49619: error is 7.9408439185
Epoch 49620: error is 8.21961225579
Epoch 49621: error is 10.5722938826
Epoch 49622: error is 9.17456191682
Epoch 49623: error is 9.96445578559
Epoch 49624: error is 10.00804987
Epoch 49625: error is 10.4491146703
Epoch 49626: error is 9.22042706306
Epoch 49627: error is 10.1529731676
Epoch 49628: error is 9.42901956127
Epoch 49629: error is 11.4759873424
Epoch 49630: error is 10.4777672443
Epoch 49631: error is 9.77258075806
Epoch 49632: error is 10.2208117629
Epoch 49633: error is 9.15245763553
Epoch 49634: error is 10.8762977227
Epoch 49635: error is 10.2369980102
Epoch 49636: error is 9.49534737981
Epoch 49637: error is 10.1205271867
Epoch 49638: error is 9.44918365034
Epoch 49639: error is 7.65075531062
Epoch 49640: error is 9.07724174

Epoch 49848: error is 8.59692936571
Epoch 49849: error is 11.0332294336
Epoch 49850: error is 10.0395995128
Epoch 49851: error is 9.15827239354
Epoch 49852: error is 8.90443428467
Epoch 49853: error is 10.3034290939
Epoch 49854: error is 10.9198892547
Epoch 49855: error is 9.53996643199
Epoch 49856: error is 7.96488240434
Epoch 49857: error is 10.7942095282
Epoch 49858: error is 8.64094413053
Epoch 49859: error is 10.0704700444
Epoch 49860: error is 8.94368053228
Epoch 49861: error is 8.98298969488
Epoch 49862: error is 9.36731532673
Epoch 49863: error is 9.31945868014
Epoch 49864: error is 10.6489812874
Epoch 49865: error is 8.11490490278
Epoch 49866: error is 11.5864823179
Epoch 49867: error is 11.2130699201
Epoch 49868: error is 8.23787741156
Epoch 49869: error is 10.4505013154
Epoch 49870: error is 10.0214391366
Epoch 49871: error is 11.7930926204
Epoch 49872: error is 10.6956109623
Epoch 49873: error is 8.31155218056
Epoch 49874: error is 11.0903529298
Epoch 49875: error is 8.4532

In [33]:
for i in range(25):
    print(test_imgs[i:i+1])

[[ 0.00605662  0.01250977  0.02142074  0.03003618  0.03206752  0.03263614
   0.02717434  0.02036579  0.01349365  0.00960305  0.00708538  0.00360255
   0.00501663  0.01040361  0.02314158  0.0336462   0.04063432  0.03728242
   0.03276708  0.02367654  0.01534542  0.00875011  0.00507648  0.00416743
   0.00464627  0.01056821  0.02020493  0.03283067  0.03883866  0.03957937
   0.03181313  0.02252433  0.01398745  0.01099843  0.00826752  0.00557403
   0.0060828   0.01069541  0.02003285  0.0316822   0.03875261  0.03743954
   0.02971819  0.02252058  0.01559981  0.01101339  0.00613143  0.00467246
   0.00588453  0.0113613   0.02047428  0.03044768  0.0370617   0.0397926
   0.03399037  0.02317151  0.01341883  0.00840594  0.00710034  0.0060005
   0.00670006  0.01207208  0.02290216  0.03398663  0.03963922  0.03662027
   0.0284313   0.0218098   0.01632929  0.01137626  0.00716768  0.00519245
   0.00652424  0.01130892  0.02177987  0.03282693  0.03788097  0.03360505
   0.026385    0.01989817  0.01440644  0

[[ 0.00274212  0.00309377  0.00511015  0.00725746  0.01016045  0.01024649
   0.01033627  0.00833112  0.00610151  0.00369233  0.00227825  0.00138416
   0.00179192  0.00243162  0.00308629  0.00458268  0.00674869  0.00844335
   0.01006318  0.01019786  0.00812537  0.00579101  0.00403276  0.00325464
   0.00225954  0.002484    0.00274961  0.00463879  0.00829745  0.01184762
   0.01413335  0.01255092  0.00866781  0.00410009  0.00282817  0.00219969
   0.00223709  0.00265608  0.00435822  0.00643071  0.01061685  0.01373307
   0.01542772  0.01373307  0.01084879  0.00610525  0.00390182  0.0029778
   0.00244659  0.0028132   0.00489692  0.00794954  0.01199352  0.01455982
   0.0153529   0.01320933  0.00900075  0.00566381  0.00378585  0.00307507
   0.00187048  0.00321349  0.00533087  0.00860421  0.01328415  0.01696526
   0.01799776  0.01504989  0.01072159  0.00759041  0.00426095  0.00337809
   0.0017171   0.002585    0.00524109  0.00957686  0.01657994  0.02114391
   0.02128607  0.01610484  0.00989858  

In [11]:
res = np.zeros(250).reshape(25,10)
for i in range(25):
    for j in range(10):
        res[i,j] = 1 - error(latent[i:i+1], test_ecfp[j:j+1])

In [12]:
np.savetxt('result.txt', res, fmt='%f')

In [34]:
for i in range(25):
    print(latent[i:i+1])

[[ 1.  0.  0.  1.  1.  1.  1.  0.  0.  0.  0.  1.  0.  0.  0.  1.  1.  0.
   1.  1.  0.  1.  0.  1.  1.  1.  0.  1.  0.  1.  0.  0.  1.  0.  0.  1.
   0.  0.  0.  1.  0.  0.  0.  1.  0.  1.  1.  1.  0.  1.  1.  0.  1.  0.
   1.  0.  0.  0.  0.  0.  1.  0.  1.  1.  1.  1.  1.  1.  1.  0.  0.  0.
   0.  0.  1.  0.  0.  1.  0.  1.  0.  0.  1.  0.  0.  1.  1.  0.  0.  1.
   1.  0.  0.  1.  0.  0.  1.  0.  1.  0.  1.  0.  1.  0.  0.  0.  1.  0.
   1.  0.  1.  0.  1.  0.  1.  1.  0.  1.  1.  1.  0.  0.  0.  0.  1.  0.
   1.  0.]]
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0. 

In [7]:
def error(a,b):
    
    if a.shape != b.shape:
        print(false)
        return 
    
    err = 0
    total = 0
    
    for i in range(a.shape[0]):
        for j in range(a.shape[1]):
            total+=1
            if a[i][j]!=b[i][j]:
                err+=1
                
    return err/total

In [59]:
for i in range(32):
    print("the difference between ECFPs row %d and the generated signal information row %d is %f" %(i,i, error(test0[i:i+1], ecfp[i:i+1])))
    

the difference between ECFPs row 0 and the generated signal information row 0 is 0.703125
the difference between ECFPs row 1 and the generated signal information row 1 is 0.710938
the difference between ECFPs row 2 and the generated signal information row 2 is 0.664062
the difference between ECFPs row 3 and the generated signal information row 3 is 0.687500
the difference between ECFPs row 4 and the generated signal information row 4 is 0.656250
the difference between ECFPs row 5 and the generated signal information row 5 is 0.593750
the difference between ECFPs row 6 and the generated signal information row 6 is 0.703125
the difference between ECFPs row 7 and the generated signal information row 7 is 0.765625
the difference between ECFPs row 8 and the generated signal information row 8 is 0.679688
the difference between ECFPs row 9 and the generated signal information row 9 is 0.781250
the difference between ECFPs row 10 and the generated signal information row 10 is 0.601562
the diff

In [13]:
print("the accuracy of the signal information generated by RBM is %f" %(1 - error(test0, ecfp)))

the accuracy of the signal information generated by RBM is 0.313232


In [37]:
error(latent[:10], test_ecfp)

0.43046875